**PROBLEM 1: Topic Models**

Obtain Topic Models (K=10, 20, 50) for both datasets by running LDA and NMF methods; you can call libraries for both methods and dont have to use the ES index as source. For both LDA and NMF: print out for each topic the top 20 words (with probabilities)

The rest of of topic exercises and results are required only for the LDA topics:
- 20NG: how well the topics align with the 20NG label classes? This is not asking for a measurement, but rather for a visual inspection to determine what topics match well with what classes. Does this change if one increases the topics from 20 to 50?

**PROBLEM 2: Extractive Summarization**

Implement the KL-Sum summarization method for each dataset. Follow the ideas in this paper ; you are allowed to use libraries for text cleaning, segmentation into sentences, etc. In this problem PS stands for "growing" summary distribution, while PD stands for fixed document distribution. Run it twice :
A) PS and PD are over words, proportional to counts of words
B) PD and PS distributions over topics, instead of distributions over words. LDA will give you the PD topic distribution at training; for PS you can call LDA[summary] (without retraining, treat summary as "new_doc")

For DUC dataset evaluate against human gold summaries with ROUGE. ROUGE Perl package. Use the "Abstract" part of the files ins folder "Summaries" as the gold summaries.
